In [1]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, cv, Pool
from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 10000)

In [2]:
with open('train_iI26erW/train.csv', 'r') as fp, open('test_plBmD8c.csv') as fg:
    train = pd.read_csv(fp)
    test = pd.read_csv(fg)

/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (8,11,12,13,14,15,16,17,18,19,21,23,26,28,29,30,32,82,83,85,91,269,325,326,327,328) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (8,11,12,13,14,15,16,17,18,19,26,28,29,30,82,83,85,91,269,325,326,327,328) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
misCols = train.isnull().sum(axis=0)/train.shape[0]

toremove = []

for k, v in misCols.iteritems():
    if v >= 0.9:
        toremove.append(k)
        
train.drop(toremove, inplace=True, axis=1)
test.drop(toremove, inplace=True, axis=1)        

In [4]:
## impute missing values
## objects with 'not_found', numerics with '-1'

for c in train.columns:
    if train[c].dtype == 'object' and train[c].isnull().any():
        train[c].fillna('MISSING', inplace=True)
        test[c].fillna('MISSING', inplace=True)
    elif (train[c].dtype == np.int64 or train[c].dtype == np.float64) and train[c].isnull().any():
        train[c].fillna(-1, inplace=True)
        test[c].fillna(-1, inplace=True)

In [5]:
train['RESPONDERS'] = train['RESPONDERS'].replace(to_replace = {'Y':1, 'N':0})

In [6]:
target = train['RESPONDERS'].values

In [8]:
trainX = train.drop(['CUSTOMER_ID','RESPONDERS'], axis=1)
testX = test.drop('CUSTOMER_ID', axis=1)

In [9]:
## get index of categorical columns
## 47 columns
catCols = [trainX.columns.get_loc(k) for k in trainX.columns if trainX[k].dtype == 'object']

In [19]:
param = {
    'use_best_model':True,
    'loss_function':'CrossEntropy',
    'eval_metric':'AUC',
    'iterations':100,
    'depth':6,
    'learning_rate':0.03,
    'rsm':0.7, #colsamlpebytree
    'random_seed':2017,
    'l2_leaf_reg':0
    
}

In [12]:
my_dt = Pool(trainX, label=target, cat_features=catCols, thread_count=4)
cb_cv = cv(param, my_dt, fold_count=5, random_seed=2017, enable_early_stopping=True, eval_period=30)

In [21]:
cb_cv

defaultdict(list,
            {"b'AUC'_test_avg": [0.6159798206301703],
             "b'AUC'_test_stddev": [0.013431690575824745],
             "b'AUC'_train_avg": [0.6166406169450166],
             "b'AUC'_train_stddev": [0.00321807074165493]})

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X_train, X_valid, y_train, y_valid = train_test_split(trainX, target, train_size = 0.7, random_state = 42)

In [36]:
#default
model = CatBoostClassifier(custom_loss=['CrossEntropy'], eval_metric = "AUC",random_seed = 42)

In [37]:
model.fit(X_train, y_train, cat_features=catCols, eval_set=(X_valid, y_valid), verbose=True, use_best_model=True)

Borders generated
feature 3 is redundant categorical feature, skipping it
0:	learn 0.6748076673	test 0.6727293413	bestTest 0.6727293413	passed: 5.57 sec	total: 30.6s	remaining: 4h 14m 26s
1:	learn 0.7194141475	test 0.7236928733	bestTest 0.7236928733	passed: 1.34 sec	total: 31.9s	remaining: 2h 12m 30s
2:	learn 0.7306781719	test 0.7329391976	bestTest 0.7329391976	passed: 1.19 sec	total: 33.1s	remaining: 1h 31m 27s
3:	learn 0.7306781719	test 0.7329391976	bestTest 0.7329391976	passed: 0.545 sec	total: 33.7s	remaining: 1h 9m 35s
4:	learn 0.7520656863	test 0.7530556121	bestTest 0.7530556121	passed: 1.12 sec	total: 34.8s	remaining: 57m 23s
5:	learn 0.7510017377	test 0.7518837952	bestTest 0.7530556121	passed: 2.08 sec	total: 36.9s	remaining: 50m 35s
6:	learn 0.7510017377	test 0.7518837952	bestTest 0.7530556121	passed: 0.594 sec	total: 37.5s	remaining: 43m 58s
7:	learn 0.7583010422	test 0.7608218221	bestTest 0.7608218221	passed: 3.14 sec	total: 40.6s	remaining: 41m 37s
8:	learn 0.7582982186	tes

In [38]:
preds1 = model.predict_proba(testX)[:,1]

In [47]:
sub1 = pd.read_csv('sample_submission_ROqqAAN.csv')

In [48]:
sub1['CUSTOMER_ID'] = test['CUSTOMER_ID']
sub1['RESPONDERS'] = preds1

In [49]:
sub1.head()

,CUSTOMER_ID,RESPONDERS
0,15800,0.011642
1,463196,0.008007
2,309959,0.000932
3,102348,0.000889
4,114850,0.014943


In [51]:
sub1.to_csv('submissions/catboost_baseline.csv', index=False) # 0.5669700910 ( best score )

In [11]:
### Model 2

#default
model = CatBoostClassifier(custom_loss=['CrossEntropy'], ## changed AUC to cross entropy 
                           eval_metric = "AUC",
                           random_seed = 2017, 
                           learning_rate=0.1, 
                           iterations=1000, 
                           thread_count = 4,
                           rsm = 0.7,
                           depth = 5,
                           od_wait = 20,
                           od_type = 'Iter'
                          )

In [15]:
model.fit(X_train, y_train, cat_features=catCols, eval_set=(X_valid, y_valid), verbose=True, use_best_model=True)

Borders generated
feature 3 is redundant categorical feature, skipping it
0:	learn 0.7582929779	test 0.7550263241	bestTest 0.7550263241	passed: 2.71 sec	total: 23.1s	remaining: 6h 24m 43s
1:	learn 0.7582929779	test 0.7550263241	bestTest 0.7550263241	passed: 0.405 sec	total: 23.5s	remaining: 3h 15m 32s
2:	learn 0.7605653	test 0.7577705685	bestTest 0.7577705685	passed: 0.703 sec	total: 24.2s	remaining: 2h 14m 7s
3:	learn 0.7631298551	test 0.7602597048	bestTest 0.7602597048	passed: 2.01 sec	total: 26.2s	remaining: 1h 48m 49s
4:	learn 0.760043518	test 0.757802964	bestTest 0.7602597048	passed: 2.31 sec	total: 28.5s	remaining: 1h 34m 38s
5:	learn 0.7606570844	test 0.7588613486	bestTest 0.7602597048	passed: 2.25 sec	total: 30.8s	remaining: 1h 24m 59s
6:	learn 0.7856755411	test 0.7812706544	bestTest 0.7812706544	passed: 3.29 sec	total: 34.1s	remaining: 1h 20m 33s
7:	learn 0.7839415979	test 0.7797458593	bestTest 0.7812706544	passed: 1.15 sec	total: 35.2s	remaining: 1h 12m 48s
8:	learn 0.8117380

In [16]:
preds1 = model.predict_proba(testX)[:,1]

In [58]:
sub1 = pd.read_csv('sample_submission_ROqqAAN.csv')
sub1['CUSTOMER_ID'] = test['CUSTOMER_ID']
sub1['RESPONDERS'] = preds1

In [59]:
sub1.to_csv('submissions/catboost_tuned.csv', index=False) ## 57

In [17]:
## this submissions is with loss function crossentropy

sub1 = pd.read_csv('sample_submission_ROqqAAN.csv')
sub1['CUSTOMER_ID'] = test['CUSTOMER_ID']
sub1['RESPONDERS'] = preds1

In [19]:
sub1.to_csv('submissions/cb_crossentropy.csv', index=False) ## 56.95 (AUC is better)

In [87]:
## upsample data in xgboost
from sklearn.model_selection import StratifiedKFold
kfold = 5
skf = StratifiedKFold(n_splits = kfold, random_state = 2018)
increase = True
for i, (train_index, test_index) in enumerate(skf.split(trainX, target)):
    print('[Fold: %d/%d]'%(i+1, kfold))
    X_train, X_valid = trainX.iloc[train_index], trainX.iloc[test_index]
    y_train, y_valid = target.iloc[train_index], target.iloc[test_index]
    
    if increase:
        pos = pd.Series(target == 1)
        
        X_train = pd.concat([X_train, X_train[pos]], axis=0)
        y_train = pd.concat([y_train, y_train.loc[pos]], axis=0)
        
        idx = np.arange(len(X_train))
        np.random.shuffle(idx)
        X_train = X_train.iloc[idx]
        y_train = y_train.iloc[idx]
        
    model.fit(X_train, y_train, cat_features=catCols, eval_set=(X_valid, y_valid), verbose=True, use_best_model=True)
        
    print('[Fold %d/%d Prediction: ]'%(i+1, kfold))
    
    p_test = model.predict_proba(testX)[:,1]
    sub1['RESPONDERS'] += p_test/kfold


[Fold: 1/5]


/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


Borders generated
feature 3 is redundant categorical feature, skipping it
0:	learn 0.7887753229	test 0.7504740872	bestTest 0.7504740872	passed: 3.34 sec	total: 30.9s	remaining: 51m 4s
1:	learn 0.7984237413	test 0.757496764	bestTest 0.757496764	passed: 1.85 sec	total: 32.8s	remaining: 26m 47s
2:	learn 0.8029657114	test 0.7627884626	bestTest 0.7627884626	passed: 2.51 sec	total: 35.3s	remaining: 19m 1s
3:	learn 0.8018292537	test 0.7609997995	bestTest 0.7627884626	passed: 1.13 sec	total: 36.4s	remaining: 14m 34s
4:	learn 0.8109598622	test 0.7757739671	bestTest 0.7757739671	passed: 3.72 sec	total: 40.2s	remaining: 12m 43s
5:	learn 0.8109598622	test 0.7757739671	bestTest 0.7757739671	passed: 0.486 sec	total: 40.6s	remaining: 10m 36s
6:	learn 0.856575882	test 0.7853198337	bestTest 0.7853198337	passed: 3.6 sec	total: 44.2s	remaining: 9m 47s
7:	learn 0.8628752507	test 0.7859878178	bestTest 0.7859878178	passed: 4.04 sec	total: 48.3s	remaining: 9m 15s
8:	learn 0.8718235619	test 0.7850558274	bestT

In [81]:
sub1.head()

,CUSTOMER_ID,RESPONDERS
0,15800,NaN
1,463196,NaN
2,309959,NaN
3,102348,NaN
4,114850,NaN
